# Regression Week 5: LASSO (coordinate descent)

In this notebook, we implement our own LASSO solver via coordinate descent.
For this goal, we:
* write a function to normalize features
* implement coordinate descent for LASSO
* explore effects of L1 penalty

In [1]:
import turicreate as tc
import numpy as np # note this allows us to refer to numpy as np instead 

### Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = tc.SFrame('../data/home_data.sframe/')

# Convert 'floors' from string into integer
sales['floors'] = sales['floors'].astype(float).astype(int)

Note: Any "feature engineering" (like creating new features or adjusting existing ones) should be done directly using the SFrames as seen in the first notebook of Week 2.

### Reusing functions from previous notebooks

From Week 2, we take `get_num_data()` in order to we convert SFrames into 2D Numpy arrays for matrix manipulation.

In [3]:
"""Receives an SFrame, a list of input feature names, and the name of target feature.
    Returns a 2d numpy array of the data matrix, and the np array of the target (outcome) vector"""
def get_numpy_data(data_sframe, features, target):
    # Add constant column for the Intercept
    data_sframe['constant'] = 1
    features = ['constant'] + features
    
    # Subselect the features from the original, including constant
    features_sframe = data_sframe[features]
    
    # Convert into a numpy matrix
    feature_matrix = features_sframe.to_numpy()
    
    # Subselect the target vector and convert into numpy array
    output_sarray = data_sframe[target]
    output_array = output_sarray.to_numpy()
    
    return (feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [4]:
""" Performs the dot product between features matrix and weight vector.
    Returns a numpy array of predicted values."""
def predict_output(feature_matrix, weights):   
    # Assume feature_matrix is a numpy matrix with features as columns,
    # Weights is a numpy array of a model's coefficients.
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

## Normalizing features
In the house dataset, features vary wildly in their relative magnitude. <br>
For instance, `sqft_living` is very large overall compared to `bedrooms`. <br>
As a result, the weight for `sqft_living` would be much smaller than the weight for `bedrooms`. This is problematic because **"small" weights are dropped first as `l1_penalty` goes up**. 

To give equal considerations for all features, we need to **normalize features**: we divide each feature by its 2-norm so that the transformed feature has norm 1.

We can do this normalization easily with Numpy: let us first consider a small matrix.

Numpy provides a shorthand for computing 2-norms of each column:

In [5]:
# Init a numpy matrix of 2x3
X = np.array([[3.,5.,8.],[4.,12.,15.]])

norms = np.linalg.norm(X, axis=0) # computes norm of each column: [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print(norms)

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [6]:
print(X / norms) # applies division element-wise [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Let us write a short function called `normalize_features(feature_matrix)`, which normalizes the columns of a given feature matrix. <br>
The function returns a pair `(normalized_features, norms)`, where the second item contains the norms of the original features. <br>
These norms are used to normalize the test data just as we normalized the training data. 

In [7]:
"""Receives a numpy matrix (i.e. a 2D numpy array), 
    returns a numpy matrix with normalized feature columns, and the numpy array of 2-norms of each column"""
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    
    return (normalized_features, norms)

To test the function, run the following:

In [8]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Coordinate Descent Implementation (normalized features)

### What is Coordinate Descent?

We seek to obtain a sparse set of weights by minimizing the LASSO Cost function (Objective), given by
`Cost = RSS + L1_norm` <br>
Specifically,
```
SUM[(prediction - output)^2] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term, since we never want to push the intercept to zero.)

We cannot use gradient descent because the absolute value sign makes the cost function non-differentiable (unless we used subgradient descent). <br>
Instead, we will use **coordinate descent**. At every iteration, we will fix all weights except the `i`-th weight, and find the value of the `i`-th weight that minimizes the Cost using partial derivatives. <br>

To compute the Objective's partial derivative with respecto to the `i`-th weight, all weights other than `w[i]` are held to be constant.

In summary, we optimize one `w[i]` at a time, circling through the weights multiple times. 
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, until convergence.

### Our implementation
For this notebook, we use **cyclical coordinate descent with normalized features**. 
- We cycle through coordinates in order through the range `0:(D-1)`, assuming the features were normalized as discussed above. <br>
- The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
    where ro is given by:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```
where the sum traverses all the items of the resulting vector in brackets.

Note that for the intercept the update is simply:
```
w[0] = ro[i]
```

## The role of the L1 penalty

As an example, consider a simple model with 2 features:

In [9]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
simple_feature_matrix, output = get_numpy_data(sales, simple_features, my_output)

In [10]:
# Normalize the features
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [11]:
# We assign some random set of initial weights
weights = np.array([1., 4., 1.])

In [12]:
# Obtain predicted values
predictions = predict_output(simple_feature_matrix, weights)

#### Inspecting the values of `ro[i]`
Compute the values of `ro[i]` of each feature in the simple model, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

In [13]:
D = len(simple_feature_matrix[0])
ro = list()

for i in range(D):
    feature_col = simple_feature_matrix[:,i]
    
    temp = feature_col * (output - predictions + weights[i]*feature_col)  # returns an array
    
    ro_i = temp.sum()
    ro.append(ro_i)

ro

[79400300.03492916, 87939470.77299108, 80966698.67596565]

***QUIZ QUESTION***

Whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2` (inclusive), the weight `w[i]` is sent to zero. 
<br>
Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. 
<br>
What range of values of `l1_penalty` **would** set `w[2]` to zero but **not** `w[1]`?
<br>
`A = (ro[2]*2, (ro[1]-1)*2)`, rounded

In [14]:
endA = ro[2]*2 # min value, shoud be equal to ro[2]*2
endB = (ro[1]-1)*2 # max value, ro[1] should still be larger
print("{:e}".format(endA),"," , "{:e}".format(endB))

if ro[1] <= endA/2.0: print("w[1] to zero A")
if ro[2] <= endA/2.0: print("w[2] to zero A")
    
if ro[1] <= endB/2.0: print("w[1] to zero B")
if ro[2] <= endB/2.0: print("w[2] to zero B")

ro[2]*2
ro[1]*2

1.619334e+08 , 1.758789e+08
w[2] to zero A
w[2] to zero B


175878941.54598215

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [15]:
if ro[1] < 175878941.546/2: print("w[1] to zero")
if ro[2] < 175878941.546/2: print("w[2] to zero")
# 175878941.546 and up

w[1] to zero
w[2] to zero


#### Thus, we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely the i-th feature is to be retained.

### Single Coordinate Descent Step

#### Implement coordinate descent that minimizes the cost function over a single feature i. 

In [16]:
"""Receives a feature matrix, output vector, current weights, l1 penalty, and index of feature to optimize over. 
   Computes and returns the new weight for feature i (ith weight)."""
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    predictions = predict_output(feature_matrix, weights)
    
    # ro is computed based on the prediction made without using the i-th weight
    feature_col = feature_matrix[:,i] 
    temp = feature_col * (output - predictions + weights[i]*feature_col)
    ro_i = temp.sum()

    if i == 0: 
        # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [17]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                    np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


## Cyclical coordinate descent 

Using the previous helper function, we can implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., up to (D-1) in order and repeat.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Finally, return weights


In [18]:
from math import inf

"""Performs cyclical coordinate descent given a feature matrix NxD, target output vector, initial weights vector,
    l1 penalty (lambda), and a stopping tolerance.
    Returns the array of optimal weights"""
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    D = len(initial_weights)
    weights = np.copy(initial_weights)
    converged = False
    it = 0
    
    while (not converged):
        it+=1
        # Reset iteration's change
        max_change=0
        # Update all weights
        for i in range(D):
            old_w_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change = abs(old_w_i - weights[i])
            max_change = max(max_change, change)  # keep track of max change in one pass
        
        converged = max_change < tolerance
    
    print(f"After {it} iterations")
    print(weights)
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [19]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

Create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [20]:
simple_feature_matrix, output = get_numpy_data(sales, simple_features, my_output)
 # normalize features
normalized_simple_feature_matrix, simple_norms = normalize_features(simple_feature_matrix)

Run our LASSO CD implementation:

In [21]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

After 93 iterations
[21624998.36636292 63157246.78545421        0.        ]


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [23]:
# Compute RSS of the model on the normalized dataset
simple_predictions = predict_output(normalized_simple_feature_matrix, weights)
residuals = output - simple_predictions
rss = (residuals**2).sum()
rss_print = "{:e}".format(rss)
print(f"RSS: {rss_print}")

RSS: 1.630492e+15


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [25]:
train_data,test_data = sales.random_split(.8, seed=0)

Let us consider the following set of features.

In [31]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']
len(all_features) + 1  # including the intercept

14

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [32]:
train_matrix, train_output = get_numpy_data(train_data, all_features, 'price')
# Normalize features
normalized_train_matrix, train_norms = normalize_features(train_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [33]:
init_weights = np.zeros(14)
tolerance=1
l1_penalty=1e7

weights1e7 = lasso_cyclical_coordinate_descent(normalized_train_matrix, train_output,
                                            init_weights, l1_penalty, tolerance)

***QUIZ QUESTION***

What features had non-zero weight in this case?
<br> A:
- intercept
- sqft_living
- waterfront
- view

In [37]:
col_names = ['intercept'] + all_features
print("Features:", len(col_names))

for i in range(len(col_names)):
    if weights1e7[i] != 0.:
        print(f"{col_names[i]} : {weights1e7[i]}")

Features: 14
intercept : 24429600.609333135
sqft_living : 48389174.35227978
waterfront : 3317511.162719814
view : 7329961.984896399


Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [40]:
init_weights = np.zeros(14)
tolerance=1
l1_penalty=1e8

weights1e8 = lasso_cyclical_coordinate_descent(normalized_train_matrix, train_output,
                                            init_weights, l1_penalty, tolerance)

***QUIZ QUESTION***

What features had non-zero weight in this case?
<br>A:
- intercept

In [41]:
col_names = ['intercept'] + all_features
print("Features:", len(col_names))

for i in range(len(col_names)):
    if weights1e8[i] != 0.:
        print(f"{col_names[i]} : {weights1e8[i]}")

Features: 14
intercept : 71114625.75280938


Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [42]:
init_weights = np.zeros(14)
tolerance=5e5
l1_penalty=1e4

weights1e4 = lasso_cyclical_coordinate_descent(normalized_train_matrix, train_output,
                                            init_weights, l1_penalty, tolerance)

***QUIZ QUESTION***

What features had non-zero weight in this case?
<br>A: all of them (14)

In [44]:
col_names = ['intercept'] + all_features
print("Features:", len(col_names))

for i in range(len(col_names)):
    if weights1e4[i] != 0.:
        print(f"{col_names[i]} : {weights1e4[i]}")

Features: 14
intercept : 77779073.91265233
bedrooms : -22884012.250233646
bathrooms : 15348487.080899928
sqft_living : 92166869.69883084
sqft_lot : -2139328.082427805
floors : -8818455.544094978
waterfront : 6494209.733106552
view : 7065162.050531973
condition : 4119079.210067598
grade : 18436483.52618777
sqft_above : -14566678.545143452
sqft_basement : -5528348.751794279
yr_built : -83591746.20730534
yr_renovated : 2784276.4601285765


### Rescaling learned weights

Our weights were computed on a normalized dataset.
To use these weights on a test set, we would have to normalize the test data too.

we can alternatively **rescale the learned weights** to include the normalization, **so we don't have to worry about normalizing any dataset anymore**.

We will scale our resulting weights so that we can make predictions with *original* features:
Computing the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without having to noramlize it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [48]:
# normalized_train_matrix, train_norms = normalize_features(train_matrix)

weights1e4_n = weights1e4 / train_norms
weights1e7_n = weights1e7 / train_norms
weights1e8_n = weights1e8 / train_norms

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [50]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [55]:
# Model with l1 = 1e4 
predictions = predict_output(test_feature_matrix, weights1e4_n)
residuals = test_output - predictions
rss = (residuals**2).sum()
print_rss = "{:e}".format(rss)
print(f"Model 1e4 RSS: {print_rss}")

Model 1e4 RSS: 2.277810e+14


In [56]:
# Model with l1 = 1e7
predictions = predict_output(test_feature_matrix, weights1e7_n)
residuals = test_output - predictions
rss = (residuals**2).sum()
print_rss = "{:e}".format(rss)
print(f"Model 1e7 RSS: {print_rss}")

Model 1e7 RSS: 2.759621e+14


In [57]:
# Model with l1 = 1e8
predictions = predict_output(test_feature_matrix, weights1e8_n)
residuals = test_output - predictions
rss = (residuals**2).sum()
print_rss = "{:e}".format(rss)
print(f"Model 1e8 RSS: {print_rss}")

Model 1e8 RSS: 5.371662e+14


***QUIZ QUESTION***

Which model performed best on the test data?
<br>
A: Model 1e4